<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Solving the Lego Planning Problem with Pyomo and Glpk</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. Based on a project at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

# 2. Solving the Lego Planning Problem

In this notebook, we explain how to solve **Linear Programming** problem that we have written to model the Lego Planning problem (see the slides on KIRO):

$$
\begin{align}
\max \quad & 8 c + 11 t  \\
 \quad & 2c + 2t \leq 24 \\
& c + 2 t \leq 18\\
& c \geq 0\\
& t \geq 0
\end{align}
$$

We show below how to use Pyomo to define the **variables**, the **objective function**, and the **constraints**.

First, we need to install the [Pyomo](http://www.pyomo.org/) modeling language and the [GLPK](https://www.gnu.org/software/glpk/) open source solver, as explained in the following section.

## 2.1 Software Installation
If you are running this notebook in a Colab, you don't need to install anything else on your computer.

Otherwise, if you have installed the recommended Anaconda Python distribution, you have to run the following two commands:

1. To install the [Pyomo](http://www.pyomo.org/) optimization modeling language:

```
conda install -c conda-forge pyomo
```

2. To install the open source [GLPK](https://www.gnu.org/software/glpk/) solver:

```
conda install -c conda-forge glpk
```

3. (Optional) You can install some extra packages of Pyomo using the following command:

```
conda install -c conda-forge pyomo.extras
```

For details about the Pyomo installation, we refer to the official [Pyomo Documentation](https://pyomo.readthedocs.io/en/stable/).

The following lines are for running this notebook in a Google Colab:

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

## 2.2 Define the model entities
To build the Linear Programming model with Pyomo, we need first to import the Pyomo library:

In [1]:
from pyomo.environ import *

At this point, we first declare a global object that refer to our model:

In [2]:
model = ConcreteModel()

Notice the `ConcreteModel` is a python class, and we are initializing an object called `model` of type `ConcreteModel`.

Then we star by declaring the two non/negative variables:

In [3]:
# declare decision variables
model.c = Var(domain=NonNegativeReals)
model.t = Var(domain=NonNegativeReals)

Here, we add variable `c` and `t` to the model. The two variables are of type `Var`, and we are passing as input to the constructor the parameter `NonNegativeReals`. That is, we are declaring the two variables $c \geq 0, t \geq 0$.

Next, we declare the objective function.

In [4]:
# Declare objective
model.cost = Objective(
    expr =  8*model.c + 11*model.t,
    sense = maximize)

In this case, we are adding the variable `cost`  to our model, that is an instance of an object of type `Objective`. The constructor is called with a linear expression `8*model.c + 11*model.t`, and we denote the sense of the objective function as `maximize`.

Next step is to introduce the two linear constraints:

In [5]:
# Declare constraints
model.cnstr1 = Constraint(expr = 2*model.c + 2*model.t <= 24)
model.cnstr2 = Constraint(expr = 1*model.c + 2*model.t <= 18)

We have used here a third type of objects, namely the `Constraint` class, giving in input the two linear relation.

Notice that we are **declaring** the model, without programming any algorithm to actually solve this model. To find the optimal solution of this LP, we are going to use the Integer Linear Programming solver GLPK.

## 2.3 Solve the model
We have use the type of objects of the Pyomo library to *declare* our Linear Programming model. Next, we have to use a solver to find the optima value for the decision variables.

With Pyomo, we are not constrained to use a specific solver. In this notebook, for the sake of simplicity, we use the GLPK solver as follows, where we set the parameter `tee=True`, in order to see the logs produced by the solver.

In [7]:
# Solver path
solvername='glpk'
# WARNING: The following path depends on your file system (windows, unix, mac)
# On Colab you can omit the executable paramater
solverpath_folder='/Users/gualandi/solvers/glpk/bin/' 
solverpath_exe=solverpath_folder+'glpsol'

# Solve the LP model
#sol = SolverFactory('glpk').solve(model, tee=True)

sol = SolverFactory(solvername, executable=solverpath_exe).solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/8l/pw0xts197510cxsrqkcgdqdh0000gn/T/tmpjba2qr23.glpk.raw
 --wglp /var/folders/8l/pw0xts197510cxsrqkcgdqdh0000gn/T/tmpc85s2k1d.glpk.glp
 --cpxlp /var/folders/8l/pw0xts197510cxsrqkcgdqdh0000gn/T/tmpwo9316l8.pyomo.lp
Reading problem data from '/var/folders/8l/pw0xts197510cxsrqkcgdqdh0000gn/T/tmpwo9316l8.pyomo.lp'...
3 rows, 3 columns, 5 non-zeros
26 lines were read
Writing problem data to '/var/folders/8l/pw0xts197510cxsrqkcgdqdh0000gn/T/tmpc85s2k1d.glpk.glp'...
19 lines were written
GLPK Simplex Optimizer 5.0
3 rows, 3 columns, 5 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =   1.140000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND


Every time we invoke a solver, it is very good practice to check the status of the solver, since it may have stop its execution for several different reasons:

In [8]:
# Basic info about the solution process
for info in sol['Solver']:
    print(info)


Status: ok
Termination condition: optimal
Statistics: 
  Branch and bound: 
    Number of bounded subproblems: 0
    Number of created subproblems: 0
Error rc: 0
Time: 0.004017829895019531



Whenever the status of the solver is OK, we can query the solver to get the optimal value of the objective function and of the decision variables.

In [9]:
# Report solution value
print("Optimal solution value: z =", model.cost())
print("Decision variables:")
print("\tProduction of chairs:", model.c())
print("\tProduction of tables:", model.t())

Optimal solution value: z = 114.0
Decision variables:
	Production of chairs: 6.0
	Production of tables: 6.0


As we found manually during the *Active Learning* session, the optimal production plan is to build 6 chairs and 6 tables, to get a profit of 114 kroner.

It is also possible to check the file temporary file produced by Pyomo and passed as input to the Glpk solver, with the following command:

In [10]:
model.write('lego1.lp')

('lego1.lp', 4882522192)

## 2.4 Complete Script
We report below the complete script.

In [ ]:
# Main model object
model = ConcreteModel()

# Declare decision variables
model.c = Var(domain=NonNegativeReals)
model.t = Var(domain=NonNegativeReals)

# Declare objective
model.cost = Objective(
    expr =  8*model.c + 11*model.t,
    sense = maximize)

# Declare constraints
model.cnstr1 = Constraint(expr = 2*model.c + 2*model.t <= 24)
model.cnstr2 = Constraint(expr = 1*model.c + 2*model.t <= 18)

# Solve the LP model
sol = SolverFactory('glpk').solve(model, tee=True)

# Basic info about the solution process
for info in sol['Solver']:
    print(info)
    
# Report solution value
print("Optimal solution value: z =", model.cost())
print("Decision variables:")
print("\tProduction of chairs:", model.c())
print("\tProduction of tables:", model.t())

## 2.5 Exercise: three variables
As an exercise, you can try to modify the previous script to solve the second version of the Lego Planning problem:

$$
\begin{align}
\max \quad & 8 c + 11 t + 15 s \\
 \quad & 2c + 2t +2s \leq 24 \\
& c + 2 t +3s \leq 18\\
& c \geq 0\\
& t \geq 0\\
& s \geq 0
\end{align}
$$

Basically, you have to add a third variable to the model, to modify the objective function, and the two constraints. Later, you call the solver again.

In [ ]:
# TO BE COMPLETED ...